In [3]:
%load_ext autoreload
%autoreload 2
import numpy as np
import xarray as xr
from matplotlib import pyplot as plt
import pandas as pd
import sys
sys.path.append('/home/onno/Thesis/Scripts')
import my_tools
from my_tools import plot_dic, file_dic
%matplotlib qt
import matplotlib.dates as mdates

In [4]:
#set path
path = '/media/onno/Volume/GFS_T850/'

Calculate Mean Absolute Error for every single vent on lead days 1, 3, 5, 7 and 9

In [5]:
coordinatez = [
(62,54,0,8),#Germany
(54,46,0,8), #Spain
(46,38,0,8), #Ukraine
(62,54,8,16),#Germany
(54,46,8,16), #Spain
(46,38,8,16), #Ukraine
(62,54,16,24),#Germanycw_lon_0_8_lat_62_54.txt
(54,46,16,24), #Spain
(46,38,16,24)] #Ukraine
#loading GFS and ERA5 data for specific box 




#Choose Forecast model: GFS or ERA5RF
fcst_modelz = ['ERA5RF']
for fcst_model in fcst_modelz:
    for lat_0,lat_1,lon_0,lon_1 in coordinatez:
        file_fcst = file_dic['T850_grid'][fcst_model].format(lon_0,lon_1-2,lat_0-2,lat_1)
        file_rean = file_dic['T850_grid']['ERA5'].format(lon_0,lon_1-2,lat_0-2,lat_1)
        file_clim_p90 = 'era51_mars_t850_79-19_24hourly_90p_lon_{}_{}_lat_{}_{}_SMOOTHED_ALL.nc'.format(lon_0,lon_1-2,lat_0-2,lat_1)
        file_clim_p50 = 'era51_mars_t850_79-19_24hourly_50p_lon_{}_{}_lat_{}_{}_SMOOTHED_ALL.nc'.format(lon_0,lon_1-2,lat_0-2,lat_1)
        file_clim_p10 = 'era51_mars_t850_79-19_24hourly_10p_lon_{}_{}_lat_{}_{}_SMOOTHED_ALL.nc'.format(lon_0,lon_1-2,lat_0-2,lat_1)
        fcst = xr.open_dataset(path+file_fcst,decode_times=False).squeeze()
        #set fcst time data to datetime object
        init_time_fcst = pd.Timestamp('1979-01-01')
        fcst['time'] = [pd.Timedelta(i-fcst.time.values[0],'hours')+init_time_fcst for i in fcst.time.values]
        fcst['lead'] = fcst['lead']//24
        #group forecast by lead time and take daily means
        fcst = fcst.sel(lead=slice(0,9)).groupby('lead').mean()
        rean = xr.open_dataset(path+file_rean).squeeze()
        if int(rean.time[0].dt.hour)!=0:
            rean = rean.assign_coords({'time':rean.time.values - pd.Timedelta(int(rean.time[0].dt.hour),'h')})
        #load percentile climatology. For convenience I set the data to all days in the leap year 2016. That has no further effect on the data
        clim_p90 = xr.open_dataset(path+file_clim_p90).squeeze()
        clim_p90['time']=pd.date_range('2016-01-01',"2016-12-31")
        clim_p10 = xr.open_dataset(path+file_clim_p10).squeeze()
        clim_p10['time']=pd.date_range('2016-01-01',"2016-12-31")
        #load all temperature extremes
        file_pers_hw = 'dates/pw_lon_{}_{}_lat_{}_{}.npy'.format(lon_0,lon_1,lat_0,lat_1)
        file_pers_cw = 'dates/pc_lon_{}_{}_lat_{}_{}.npy'.format(lon_0,lon_1,lat_0,lat_1)
        file_pers_avg = 'dates/avg_p40_p60_lon_{}_{}_lat_{}_{}.npy'.format(lon_0,lon_1,lat_0,lat_1)
        pers_hw = np.load(path+file_pers_hw,allow_pickle=True)
        pers_cw = np.load(path+file_pers_cw,allow_pickle=True)
        pers_avg = np.load(path+file_pers_avg,allow_pickle=True)
        temp_extremez = [pers_hw,pers_cw,pers_avg]
        #set column names for csv file
        columnz = ['persistent_hw','persistent_cw','persistent_avg',
                   'length_persistent_hw','length_persistent_cw','length_persistent_avg']
        #create index for csv file
        index = pd.date_range(pd.Timestamp('1979-01-01'),pd.Timestamp('2019-12-31'))
        #loop over lead days
        lead_dayz = [3,5,7]
        for lead_day in lead_dayz:
            #create empty pandas dataframe
            df = pd.DataFrame(index=index,columns=columnz,dtype=float)
            #Loop over all extreme events
            for j,temp_extreme in enumerate(temp_extremez):
                #Select persistent or short-lived warm or cold temp extreme
                column = columnz[j]
                #Loop over each individual event
                for i,(begin_date,end_date) in enumerate(temp_extreme):
                    begin_date = pd.Timestamp(begin_date);end_date=pd.Timestamp(end_date)
                    #If an event in the first few days of the datset requires a forecast before 1-12-84 skip this event
                    if begin_date<pd.Timestamp('1979-01-01')+pd.Timedelta(lead_day,'days'):
                        continue
                    #define date of required forecast
                    begin_lead_date = begin_date - pd.Timedelta(lead_day,'days')
                    #Skip events that occur on 29,30 or 31 Oct because these days are not present in the ERA5 reforecast
                    if np.isin(begin_lead_date,[pd.Timestamp('2014-10-29'),
                                           pd.Timestamp('2014-10-30'),
                                           pd.Timestamp('2014-10-31')]):
                        continue
                    #define date for highest lead day (last day of forecast)
                    end_lead_date = begin_date + pd.Timedelta(9-lead_day,'days')
                    #Calculate lead day of on last day of observed temperature extreme 
                    end_lead_day = (end_date - begin_lead_date).days
                    #if last day of extreme event exceeds the forecast range,
                    #I set the last day of the event on the day where the latest forecast is available
                    if (pd.Timestamp(end_date)-pd.Timestamp(begin_date)).days > (9-lead_day): 
                        rean_sub = rean.sel(time=slice(begin_date,end_lead_date)).load()
                    else:
                        rean_sub = rean.sel(time=slice(begin_date,end_date)).load()
                    #Select forecast date with subsequent lead days where event was observed
                    fcst_sub = fcst.sel(time=begin_lead_date,lead=slice(lead_day,end_lead_day)).load()
                    
                    assert(begin_lead_date+pd.Timedelta(end_lead_day,'days')==end_date)
                    #Calculate Mean Absolute Error of forecast
                    MAE = float(np.abs(fcst_sub.t-rean_sub.t.values).mean().values)
                    #The code below is a check to see whether all days in the reanalysis truly are extreme
                    if (pd.Timestamp(end_date)-pd.Timestamp(begin_date)).days > (9-lead_day):
                        timez = [pd.Timestamp('2016-{:02d}-{:02d}'.format(i.month,i.day)) for i in pd.date_range(begin_date,end_lead_date)]
                    else:
                        timez = [pd.Timestamp('2016-{:02d}-{:02d}'.format(i.month,i.day)) for i in pd.date_range(begin_date,end_date)]
                    if j==0:
                        clim_p90_sub = clim_p90.sel(time=timez).load()
                        rean_bool = rean_sub.t>clim_p90_sub.t.values
                        fcst_bool = fcst_sub.t>clim_p90_sub.t.values
                    else:
                        clim_p10_sub = clim_p10.sel(time=timez).load()
                        rean_bool = rean_sub.t<clim_p10_sub.t.values
                        fcst_bool = fcst_sub.t<clim_p10_sub.t.values
                        
                    #If this statement is true all days within the renalysis are extreme
                    if j<=1:
                        assert(rean_bool.values.all())
                    #Put all MAE values in the row with the same date as the onset of the event and in the column for the subsequent extreme
                    df.loc[begin_date][column]=MAE
                    #Make an extra column with the length of the reanalysis extreme length (corrected for forecast range)
                    df.loc[begin_date]['length_'+column]=len(fcst_sub.t)
            df = df.dropna(how='all')
            df.to_csv(path+'MAE/Mean_Absolute_Error_lead_day_{}_lon_{}_{}_lat_{}_{}_{}.csv'.format(lead_day,lon_0,lon_1,
                                                                                       lat_0,lat_1,fcst_model))
